In [1]:
from utils import *
import load_data
from Project_Models import RNN_Model
import torch
import torch.nn as nn

def normalize(x):
    maxtensor = x.max(0, keepdim=True)[0]
    maxtensor[maxtensor==0] = 1e-4
    x_normed = x / maxtensor
    return x_normed, maxtensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = load_data.load_county_level()

loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [4]:
maxload = 100

x = form_input_tensor(df, ['#Hospitals', '#ICU_beds', 'MedicareEnrollment,AgedTot2017', 'DiabetesPercentage'], maxload=maxload).to(device)
x, xmaxtensor = normalize(x)
print(f"#x nan: {(torch.sum(torch.isnan(x)))}")
print("x:", x)

y = form_labels_tensor(df, maxload=maxload).to(device)
y, ymaxtensor = normalize(y)
print(f"#y nan: {torch.sum(torch.isnan(y))}")
print("y:", y * ymaxtensor)

#x nan: 0
x: tensor([[[0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.0000, 0.0000],
         ...,
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2434, 1.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2433, 1.0000],
         [0.1579, 0.1496, 0.2425, 0.7638, 0.2426, 1.0000]],

        [[0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.0000, 0.0000],
         ...,
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2464, 0.9824],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2463, 0.9827],
         [0.0789, 0.0607, 0.2413, 0.8031, 0.2456, 0.9825]],

        [[1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         [1.0000, 1.0000, 1.0000, 0.6378, 0.0000, 0.0000],
         ...,
         [1.0000, 1.0000, 1.0000, 0.6378, 1.0000, 0.9648

In [5]:
model = RNN_Model.RNN(x.shape[2], 128, 2).to(device)
with torch.no_grad():
    print("Before training:", model(x) * ymaxtensor)

criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()
learning_rate = 1e-2
num_epoches = 800
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    outputs = model(x)

    loss = criterion(outputs, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epoches}], Loss: {loss.item():.4f}')
with torch.no_grad():
    print("After training:", (model(x) * ymaxtensor)[10:20])
    print("y:", (y * ymaxtensor)[10:20])

0276
Epoch [159/800], Loss: 0.0276
Epoch [160/800], Loss: 0.0276
Epoch [161/800], Loss: 0.0276
Epoch [162/800], Loss: 0.0276
Epoch [163/800], Loss: 0.0276
Epoch [164/800], Loss: 0.0276
Epoch [165/800], Loss: 0.0276
Epoch [166/800], Loss: 0.0276
Epoch [167/800], Loss: 0.0276
Epoch [168/800], Loss: 0.0276
Epoch [169/800], Loss: 0.0276
Epoch [170/800], Loss: 0.0276
Epoch [171/800], Loss: 0.0276
Epoch [172/800], Loss: 0.0276
Epoch [173/800], Loss: 0.0276
Epoch [174/800], Loss: 0.0276
Epoch [175/800], Loss: 0.0276
Epoch [176/800], Loss: 0.0276
Epoch [177/800], Loss: 0.0276
Epoch [178/800], Loss: 0.0276
Epoch [179/800], Loss: 0.0276
Epoch [180/800], Loss: 0.0276
Epoch [181/800], Loss: 0.0276
Epoch [182/800], Loss: 0.0276
Epoch [183/800], Loss: 0.0276
Epoch [184/800], Loss: 0.0276
Epoch [185/800], Loss: 0.0276
Epoch [186/800], Loss: 0.0276
Epoch [187/800], Loss: 0.0276
Epoch [188/800], Loss: 0.0276
Epoch [189/800], Loss: 0.0276
Epoch [190/800], Loss: 0.0276
Epoch [191/800], Loss: 0.0276
Epoch

In [7]:
model = RNN_Model.RNN(x.shape[2], 128, 2).to(device)

with torch.no_grad():
    print("Before training:", model(x) * ymaxtensor)

criterion = nn.MSELoss()
# criterion = nn.SmoothL1Loss()
learning_rate = 1e-2
num_epoches = 800
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    outputs = model(x)

    loss = criterion(outputs, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    print(f'Epoch [{epoch + 1}/{num_epoches}], Loss: {loss.item():.4f}')
with torch.no_grad():
    print("After training:", (model(x) * ymaxtensor)[10:20])
    print("y:", (y * ymaxtensor)[10:20])

0276
Epoch [159/800], Loss: 0.0276
Epoch [160/800], Loss: 0.0276
Epoch [161/800], Loss: 0.0276
Epoch [162/800], Loss: 0.0276
Epoch [163/800], Loss: 0.0276
Epoch [164/800], Loss: 0.0276
Epoch [165/800], Loss: 0.0276
Epoch [166/800], Loss: 0.0276
Epoch [167/800], Loss: 0.0276
Epoch [168/800], Loss: 0.0276
Epoch [169/800], Loss: 0.0276
Epoch [170/800], Loss: 0.0276
Epoch [171/800], Loss: 0.0276
Epoch [172/800], Loss: 0.0276
Epoch [173/800], Loss: 0.0276
Epoch [174/800], Loss: 0.0276
Epoch [175/800], Loss: 0.0276
Epoch [176/800], Loss: 0.0276
Epoch [177/800], Loss: 0.0276
Epoch [178/800], Loss: 0.0276
Epoch [179/800], Loss: 0.0276
Epoch [180/800], Loss: 0.0276
Epoch [181/800], Loss: 0.0276
Epoch [182/800], Loss: 0.0276
Epoch [183/800], Loss: 0.0276
Epoch [184/800], Loss: 0.0276
Epoch [185/800], Loss: 0.0276
Epoch [186/800], Loss: 0.0276
Epoch [187/800], Loss: 0.0276
Epoch [188/800], Loss: 0.0276
Epoch [189/800], Loss: 0.0276
Epoch [190/800], Loss: 0.0276
Epoch [191/800], Loss: 0.0276
Epoch

In [ ]:
# TODO
# 1. Batch Gradient Descent
# 2. Handle NaN case
# 3. clean up code